<a href="https://colab.research.google.com/github/spdraptor/Coursera_Capstone/blob/main/neighborhoods_in_Toronto(w3_Part2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **IBM coursera neighborhoods in Toronto**

In [1]:
# Package Instalation
!pip install beautifulsoup4
!pip install geocoder

     |████████████████████████████████| 102kB 3.1MB/s 


 **import all packages**

In [13]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import geocoder # import geocoder

**Parse the Wiki link to extract table data**

In [14]:
table_contents=[]
page=urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(page)
table=soup.find('table')
for row in table.findAll('td'):
  cell = {}
  if row.span.text=='Not assigned':
      pass
  else:
      cell['PostalCode'] = row.p.text[:3]
      cell['Borough'] = (row.span.text).split('(')[0]
      cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
      table_contents.append(cell)

**Dispaly and check the data**

In [15]:
table_contents

[{'Borough': 'North York', 'Neighborhood': 'Parkwoods', 'PostalCode': 'M3A'},
 {'Borough': 'North York',
  'Neighborhood': 'Victoria Village',
  'PostalCode': 'M4A'},
 {'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront',
  'PostalCode': 'M5A'},
 {'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights',
  'PostalCode': 'M6A'},
 {'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government',
  'PostalCode': 'M7A'},
 {'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue',
  'PostalCode': 'M9A'},
 {'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge',
  'PostalCode': 'M1B'},
 {'Borough': 'North York',
  'Neighborhood': 'Don Mills North',
  'PostalCode': 'M3B'},
 {'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens',
  'PostalCode': 'M4B'},
 {'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson',
  'PostalCode': 'M5B'},
 {'Borough': 'North York', 'Neighborhood': 'Glenca

**Convert the extracted data in DataFrame**

In [16]:
df=pd.DataFrame(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**Find the shape or the data size**

In [17]:
df.shape

(103, 3)

***geocoder.google wasn't returning anything so I used geocoder.arcgis to get coordinate***

defined function to get coordinate

In [18]:
def coordinate(postal_code):
  # initialize your variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
  return lat_lng_coords  #returns in format[latitude, longitude]


In [19]:
coordinate('M5G')

[43.65609000000006, -79.38492999999994]

In [24]:
!pip install tqdm

In [28]:
from tqdm import tqdm
temp_df = pd.DataFrame()
pc=[]
lat=[]
long=[]
for i in tqdm(df['PostalCode']):
  a=coordinate(i)
  pc.append(i)
  lat.append(a[0])
  long.append(a[1])

temp_df['PostalCode']=pc
temp_df['latitude']=lat
temp_df['longitude']=long
print('done')


100%|██████████| 103/103 [00:54<00:00,  1.89it/s]

done


In [29]:
temp_df

,PostalCode,latitude,longitude
0,M3A,43.75245,-79.32991
1,M4A,43.73057,-79.31306
2,M5A,43.65512,-79.36264
3,M6A,43.72327,-79.45042
4,M7A,43.66253,-79.39188
...,...,...,...
98,M8X,43.65319,-79.51113
99,M4Y,43.66659,-79.38133
100,M7Y,43.64869,-79.38544
101,M8Y,43.63278,-79.48945


In [30]:
df2 = df.merge(temp_df, on="PostalCode", how="left")
df2.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
